In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import ast

pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)


import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet as wn

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/thomasmanson/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/thomasmanson/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/thomasmanson/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/thomasmanson/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/thomasmanson/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [7]:
from recettes_et_sentiments.api_model import data, preprocessing


ModuleNotFoundError: No module named 'recettes_et_sentiments'

In [6]:
preprocessing.ping()


'pong'

In [3]:
!pip list | grep recette

recettes_et_sentiments       0.1         /Users/thomasmanson/code/mansonthomas-wagon/projets/recettes-et-sentiments/recettes_et_sentiments


In [31]:
my_data = data.load_data("../../batch-1672-recettes-et-sentiments-data/RAW_recipes.csv")
df = my_data[0:8000].copy()

In [32]:
df

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8
...,...,...,...,...,...,...,...,...,...,...,...,...
7995,apricot barbecued chicken,81712,45,67899,2004-01-21,"['60-minutes-or-less', 'time-to-make', 'course...","[266.6, 7.0, 61.0, 9.0, 53.0, 4.0, 9.0]",6,"['in a large skillet , cook chicken and garlic...","this looks so scrumptious and sweet, i can't w...","['boneless skinless chicken breast', 'garlic',...",8
7996,apricot barley casserole,107482,21,52282,2005-01-03,"['30-minutes-or-less', 'time-to-make', 'course...","[393.7, 23.0, 51.0, 29.0, 23.0, 23.0, 18.0]",8,"['preheat oven to 325', 'grease a 13x9 baking ...",very easy to throw together this bakes while y...,"['pine nuts', 'butter', 'pearl barley', 'scall...",7
7997,apricot bars,115225,40,68911,2005-04-04,"['60-minutes-or-less', 'time-to-make', 'course...","[212.6, 18.0, 54.0, 1.0, 10.0, 5.0, 7.0]",10,['preheat oven to 350 oven and spray 8 x 8 inc...,i always liked apricots and its high nutrition...,"['apricot', 'almonds', 'walnuts', 'honey', 'wh...",8
7998,apricot bars with shortbread crust,141910,80,89831,2005-10-19,"['time-to-make', 'course', 'main-ingredient', ...","[89.1, 5.0, 38.0, 2.0, 2.0, 8.0, 4.0]",17,"['set oven to 350 degrees', 'grease an 8x8-inc...",a moist apricot-walnut layer on top of a light...,"['flour', 'sugar', 'salt', 'butter', 'dried ap...",11


In [35]:
df = preprocessing.add_columns_and_merge_text(my_data)

In [27]:
def get_stopwords(type:int=0)->set:
    """
    Return english (stopwords) from NLTK if type=0

    TODO : support stopwords customized for our use case by passing other value for type

    Args:
        type (int): Type of stopwords to return.
            0 => default english NLTK stopwords
            Other value raise ValueError as of now

    Returns:
        Set[str]: set of stopwords

    Raises:
        ValueError: if type is not supported
    """
    stop_words = set(stopwords.words('english'))

    if type == 0:
        return stop_words

    raise ValueError(f"type {type} not supported")



def remove_stopwords(text:str, stop_words:set)->str:
    """
    Return the "text" argument without the stop_words


    Args:
        text (str): text to be processed.
        stop_words (set) : set of stop words to be used for processing the text.

    Returns:
        str: the input text without stop words
    """
    tokens = word_tokenize(text)
    tokenized_sentence_no_stopwords = [w for w in tokens if not w in stop_words]
    return ' '.join(tokenized_sentence_no_stopwords)


def lemmatize_text(text:str)->str:
    """
    Return the lemmatize "text"

    Args:
        text (str): text to be processed. Text should have been preprocessed (lowercase, ponctuation removed, stopwords removed)

    Returns:
        str: the input text without stop words
    """
    def get_wordnet_pos(treebank_tag):
        if treebank_tag.startswith('J'):
            return wn.ADJ
        elif treebank_tag.startswith('V'):
            return wn.VERB
        elif treebank_tag.startswith('N'):
            return wn.NOUN
        elif treebank_tag.startswith('R'):
            return wn.ADV
        else:
            return wn.NOUN  # Par défaut, retourne NOM

    tokens = word_tokenize(text)
    pos_tags = pos_tag(tokens)

    lemmatizer = WordNetLemmatizer()

    lemmatized_tokens = [lemmatizer.lemmatize(token, get_wordnet_pos(pos)) for token, pos in pos_tags]

    return ' '.join(lemmatized_tokens)

In [28]:
stop_words = get_stopwords()

In [36]:
df['merged_ingredients'] = df['ingredients'].apply(lambda x: "\n".join(x))


In [37]:
df

,name,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,calories,total_fat_pdv,sugar_pdv,sodium_pdv,protein_pdv,saturated_fat_pdv,carbohydrates_pdv,merged_steps,merged_steps_length,merged_ingredients
id,,,,,,,,,,,,,,,,,,,,,
137739,arriba baked winter squash mexican style,55,47892,2005-09-16,"[60-minutes-or-less, time-to-make, course, mai...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"[make a choice and proceed with recipe, depend...",autumn is my favorite time of year to cook! th...,"[winter squash, mexican seasoning, mixed spice...",7,51.5,0.0,13.0,0.0,2.0,0.0,4.0,make a choice and proceed with recipe\ndependi...,697,winter squash\nmexican seasoning\nmixed spice\...
31490,a bit different breakfast pizza,30,26278,2002-06-17,"[30-minutes-or-less, time-to-make, course, mai...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"[preheat oven to 425 degrees f, press dough in...",this recipe calls for the crust to be prebaked...,"[prepared pizza crust, sausage patty, eggs, mi...",6,173.4,18.0,0.0,17.0,22.0,35.0,1.0,preheat oven to 425 degrees f\npress dough int...,386,prepared pizza crust\nsausage patty\neggs\nmil...
112140,all in the kitchen chili,130,196586,2005-02-25,"[time-to-make, course, preparation, main-dish,...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"[brown ground beef in large pot, add chopped o...",this modified version of 'mom's' chili was a h...,"[ground beef, yellow onions, diced tomatoes, t...",13,269.8,22.0,32.0,48.0,39.0,27.0,5.0,brown ground beef in large pot\nadd chopped on...,293,ground beef\nyellow onions\ndiced tomatoes\nto...
59389,alouette potatoes,45,68585,2003-04-14,"[60-minutes-or-less, time-to-make, course, mai...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,[place potatoes in a large pot of lightly salt...,"this is a super easy, great tasting, make ahea...","[spreadable cheese with garlic and herbs, new ...",11,368.1,17.0,10.0,2.0,14.0,8.0,20.0,place potatoes in a large pot of lightly salte...,543,spreadable cheese with garlic and herbs\nnew p...
44061,amish tomato ketchup for canning,190,41706,2002-10-25,"[weeknight, time-to-make, course, main-ingredi...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,"[mix all ingredients& boil for 2 1 / 2 hours ,...",my dh's amish mother raised him on this recipe...,"[tomato juice, apple cider vinegar, sugar, sal...",8,352.9,1.0,337.0,23.0,3.0,0.0,28.0,"mix all ingredients& boil for 2 1 / 2 hours , ...",289,tomato juice\napple cider vinegar\nsugar\nsalt...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486161,zydeco soup,60,227978,2012-08-29,"[ham, 60-minutes-or-less, time-to-make, course...","[415.2, 26.0, 34.0, 26.0, 44.0, 21.0, 15.0]",7,"[heat oil in a 4-quart dutch oven, add celery ...",this is a delicious soup that i originally fou...,"[celery, onion, green sweet pepper, garlic clo...",22,415.2,26.0,34.0,26.0,44.0,21.0,15.0,"heat oil in a 4-quart dutch oven\nadd celery ,...",462,celery\nonion\ngreen sweet pepper\ngarlic clov...
493372,zydeco spice mix,5,1500678,2013-01-09,"[15-minutes-or-less, time-to-make, course, pre...","[14.8, 0.0, 2.0, 58.0, 1.0, 0.0, 1.0]",1,[mix all ingredients together thoroughly],this spice mix will make your taste buds dance!,"[paprika, salt, garlic powder, onion powder, d...",13,14.8,0.0,2.0,58.0,1.0,0.0,1.0,mix all ingredients together thoroughly,39,paprika\nsalt\ngarlic powder\nonion powder\ndr...
308080,zydeco ya ya deviled eggs,40,37779,2008-06-07,"[60-minutes-or-less, time-to-make, course, mai...","[59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0]",7,"[in a bowl , combine the mashed yolks and mayo...","deviled eggs, cajun-style","[hard-cooked eggs, mayonnaise, dijon mustard, ...",8,59.2,6.0,2.0,3.0,6.0,5.0,0.0,"in a bowl , combine the mashed yolks and mayon...",246,hard-cooked eggs\nmayonnaise\ndijon mustard\ns...


In [38]:
df['merged_steps'] = df['steps'].apply(lambda x: "\n".join(x))
df['merged_tags'] = df['tags'].apply(lambda x: "\n".join(x))

In [30]:
df

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,merged_ingredients,merged_steps,merged_tags
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,[\n'\nw\ni\nn\nt\ne\nr\n \ns\nq\nu\na\ns\nh\n'...,[\n'\nm\na\nk\ne\n \na\n \nc\nh\no\ni\nc\ne\n ...,[\n'\n6\n0\n-\nm\ni\nn\nu\nt\ne\ns\n-\no\nr\n-...
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,[\n'\np\nr\ne\np\na\nr\ne\nd\n \np\ni\nz\nz\na...,[\n'\np\nr\ne\nh\ne\na\nt\n \no\nv\ne\nn\n \nt...,[\n'\n3\n0\n-\nm\ni\nn\nu\nt\ne\ns\n-\no\nr\n-...
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,"[\n'\ng\nr\no\nu\nn\nd\n \nb\ne\ne\nf\n'\n,\n ...",[\n'\nb\nr\no\nw\nn\n \ng\nr\no\nu\nn\nd\n \nb...,"[\n'\nt\ni\nm\ne\n-\nt\no\n-\nm\na\nk\ne\n'\n,..."
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,[\n'\ns\np\nr\ne\na\nd\na\nb\nl\ne\n \nc\nh\ne...,[\n'\np\nl\na\nc\ne\n \np\no\nt\na\nt\no\ne\ns...,[\n'\n6\n0\n-\nm\ni\nn\nu\nt\ne\ns\n-\no\nr\n-...
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,"[\n'\nt\no\nm\na\nt\no\n \nj\nu\ni\nc\ne\n'\n,...",[\n'\nm\ni\nx\n \na\nl\nl\n \ni\nn\ng\nr\ne\nd...,"[\n'\nw\ne\ne\nk\nn\ni\ng\nh\nt\n'\n,\n \n'\nt..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7995,apricot barbecued chicken,81712,45,67899,2004-01-21,"['60-minutes-or-less', 'time-to-make', 'course...","[266.6, 7.0, 61.0, 9.0, 53.0, 4.0, 9.0]",6,"['in a large skillet , cook chicken and garlic...","this looks so scrumptious and sweet, i can't w...","['boneless skinless chicken breast', 'garlic',...",8,[\n'\nb\no\nn\ne\nl\ne\ns\ns\n \ns\nk\ni\nn\nl...,[\n'\ni\nn\n \na\n \nl\na\nr\ng\ne\n \ns\nk\ni...,[\n'\n6\n0\n-\nm\ni\nn\nu\nt\ne\ns\n-\no\nr\n-...
7996,apricot barley casserole,107482,21,52282,2005-01-03,"['30-minutes-or-less', 'time-to-make', 'course...","[393.7, 23.0, 51.0, 29.0, 23.0, 23.0, 18.0]",8,"['preheat oven to 325', 'grease a 13x9 baking ...",very easy to throw together this bakes while y...,"['pine nuts', 'butter', 'pearl barley', 'scall...",7,"[\n'\np\ni\nn\ne\n \nn\nu\nt\ns\n'\n,\n \n'\nb...",[\n'\np\nr\ne\nh\ne\na\nt\n \no\nv\ne\nn\n \nt...,[\n'\n3\n0\n-\nm\ni\nn\nu\nt\ne\ns\n-\no\nr\n-...
7997,apricot bars,115225,40,68911,2005-04-04,"['60-minutes-or-less', 'time-to-make', 'course...","[212.6, 18.0, 54.0, 1.0, 10.0, 5.0, 7.0]",10,['preheat oven to 350 oven and spray 8 x 8 inc...,i always liked apricots and its high nutrition...,"['apricot', 'almonds', 'walnuts', 'honey', 'wh...",8,"[\n'\na\np\nr\ni\nc\no\nt\n'\n,\n \n'\na\nl\nm...",[\n'\np\nr\ne\nh\ne\na\nt\n \no\nv\ne\nn\n \nt...,[\n'\n6\n0\n-\nm\ni\nn\nu\nt\ne\ns\n-\no\nr\n-...
7998,apricot bars with shortbread crust,141910,80,89831,2005-10-19,"['time-to-make', 'c

In [39]:
df['merged_ingredients_wo_sw'] = df['merged_ingredients'].apply(lambda x: remove_stopwords(x, stop_words=stop_words))

In [40]:
df['merged_ingredients_wo_sw_lem'] = df['merged_ingredients_wo_sw'].apply(lemmatize_text)


In [19]:
def count_vectorize(
    df:pd.DataFrame,
    column_name:str,
    min_df:float=1.0,
    max_df:float=1.0,
    max_features:int=None,
    ngram_range=(1,1)
    )->pd.DataFrame:
    """
    usage :
    CountVectorize the text in column column_name and add to the Data Frame the new columns

    Args:
        df (pd.DataFrame): DataFrame in which there is a preprocessed text column to be vectorized
        colum_name (str) : name of the column that contains the text to CountVectorize

    Returns:
        pd.DataFrame

    """

    count_vectorizer = CountVectorizer(
        min_df=min_df,
        max_df=max_df,
        max_features=max_features,
        ngram_range=ngram_range
        )

    X = count_vectorizer.fit_transform(df[column_name])
    X_df = pd.DataFrame(X.toarray(), columns = count_vectorizer.get_feature_names_out())

    return pd.concat([df, X_df], axis = 1)


In [24]:
df

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,merged_ingredients,merged_steps,merged_tags,merged_ingredients_wo_sw,merged_ingredients_wo_sw_lem
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,[\n'\nw\ni\nn\nt\ne\nr\n \ns\nq\nu\na\ns\nh\n'...,[\n'\nm\na\nk\ne\n \na\n \nc\nh\no\ni\nc\ne\n ...,[\n'\n6\n0\n-\nm\ni\nn\nu\nt\ne\ns\n-\no\nr\n-...,"[ ' w n e r q u h ' , ' e x c n e n n g ' , ' ...","[ ' w n e r q u h ' , ' e x c n e n n g ' , ' ..."
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6,[\n'\np\nr\ne\np\na\nr\ne\nd\n \np\ni\nz\nz\na...,[\n'\np\nr\ne\nh\ne\na\nt\n \no\nv\ne\nn\n \nt...,[\n'\n3\n0\n-\nm\ni\nn\nu\nt\ne\ns\n-\no\nr\n-...,"[ ' p r e p r e p z z c r u ' , ' u g e p ' , ...","[ ' p r e p r e p z z c r u ' , ' u g e p ' , ..."
2,all in the kitchen chili,112140,130,196586,2005-02-25,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13,"[\n'\ng\nr\no\nu\nn\nd\n \nb\ne\ne\nf\n'\n,\n ...",[\n'\nb\nr\no\nw\nn\n \ng\nr\no\nu\nn\nd\n \nb...,"[\n'\nt\ni\nm\ne\n-\nt\no\n-\nm\na\nk\ne\n'\n,...","[ ' g r u n b e e f ' , ' e l l w n n ' , ' c ...","[ ' g r u n b e e f ' , ' e l l w n n ' , ' c ..."
3,alouette potatoes,59389,45,68585,2003-04-14,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11,[\n'\ns\np\nr\ne\na\nd\na\nb\nl\ne\n \nc\nh\ne...,[\n'\np\nl\na\nc\ne\n \np\no\nt\na\nt\no\ne\ns...,[\n'\n6\n0\n-\nm\ni\nn\nu\nt\ne\ns\n-\no\nr\n-...,[ ' p r e b l e c h e e e w h g r l c n h e r ...,[ ' p r e b l e c h e e e w h g r l c n h e r ...
4,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8,"[\n'\nt\no\nm\na\nt\no\n \nj\nu\ni\nc\ne\n'\n,...",[\n'\nm\ni\nx\n \na\nl\nl\n \ni\nn\ng\nr\ne\nd...,"[\n'\nw\ne\ne\nk\nn\ni\ng\nh\nt\n'\n,\n \n'\nt...","[ ' j u c e ' , ' p p l e c e r v n e g r ' , ...","[ ' j u c e ' , ' p p l e c e r v n e g r ' , ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,almond encrusted english toffee,106417,45,115822,2004-12-19,"['60-minutes-or-less', 'time-to-make', 'course...","[370.5, 48.0, 68.0, 6.0, 11.0, 72.0, 7.0]",18,['chop the almonds until they are broken into ...,i got this recipe last year from the atlanta j...,"['almonds', 'unsalted butter', 'sea salt', 'va...",6,"[\n'\na\nl\nm\no\nn\nd\ns\n'\n,\n \n'\nu\nn\ns...",[\n'\nc\nh\no\np\n \nt\nh\ne\n \na\nl\nm\no\nn...,[\n'\n6\n0\n-\nm\ni\nn\nu\nt\ne\ns\n-\no\nr\n-...,"[ ' l n ' , ' u n l e b u e r ' , ' e l ' , ' ...","[ ' l n ' , ' u n l e b u e r ' , ' e l ' , ' ..."
3996,almond lemon tea cake,246823,95,37779,2007-08-16,"['time-to-make', 'course', 'preparation', 'des...","[568.6, 49.0, 209.0, 4.0, 14.0, 80.0, 21.0]",31,['position a rack in the lower third of the ov...,"tartine bakery, san francisco, ca","['cake flour', 'baking powder', 'salt', 'eggs'...",12,"[\n'\nc\na\nk

In [41]:

df2 = count_vectorize(df, 'merged_ingredients_wo_sw_lem')

[autoreload of api_model.preprocessing failed: Traceback (most recent call last):
  File "/Users/thomasmanson/.pyenv/versions/3.10.6/envs/recettes-et-sentiments/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/Users/thomasmanson/.pyenv/versions/3.10.6/envs/recettes-et-sentiments/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
  File "/Users/thomasmanson/.pyenv/versions/3.10.6/lib/python3.10/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 619, in _exec
  File "<frozen importlib._bootstrap_external>", line 883, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/Users/thomasmanson/code/mansonthomas-wagon/projets/recettes-et-sentiments/recettes_et_sentiments/api_model/preprocessing.py", line 7, in <module>
   

ValueError: After pruning, no terms remain. Try a lower min_df or a higher max_df.

In [46]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 231637 entries, 137739 to 298509
Columns: 4021 entries, name to zwieback
dtypes: datetime64[ns](1), float64(7), int64(4002), object(11)
memory usage: 6.9+ GB


In [10]:



def tfidf_vectorize(
    df:pd.DataFrame,
    column_name:str,
    min_df:float=1.0,
    max_df:float=1.0,
    max_features:int=None,
    ngram_range=(1,1))->pd.DataFrame:
    """
    usage :
    TFIDF Vectorize the text in column column_name and add to the Data Frame the new columns

    Args:
        df (pd.DataFrame): DataFrame in which there is a preprocessed text column to be vectorized
        colum_name (str) : name of the column that contains the text to Vectorize

    Returns:
        pd.DataFrame

    """
    tf_idf_vectorizer = TfidfVectorizer(
        min_df=min_df,
        max_df=max_df,
        max_features=max_features,
        ngram_range=ngram_range
        )

    X = tf_idf_vectorizer.fit_transform(df[column_name])
    X_df = pd.DataFrame(X.toarray(), columns = tf_idf_vectorizer.get_feature_names_out())

    return pd.concat([df, X_df], axis = 1)

In [57]:
df = tfidf_vectorize(df, 'merged_ingredients_wo_sw_lem')
df

,name,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,calories,total_fat_pdv,sugar_pdv,sodium_pdv,protein_pdv,saturated_fat_pdv,carbohydrates_pdv,merged_steps,merged_steps_length,merged_ingredients,merged_tags,merged_ingredients_wo_sw,merged_ingredients_wo_sw_lem,10,100,10x,12,13,15,151,16,18,21,35,43,57,70,80,85,90,92,93,94,95,96,97,98,99,abalone,...,worthington,wrap,wrapper,wraps,wyler,xanthan,xxxx,xylitol,yabbies,yacon,yak,yakisoba,yam,yams,yard,yautia,yeast,yellow,yellowfin,yellowtail,yigandes,yoghurt,yogurt,yolk,yolks,yoplait,york,yoshida,young,yucca,yukon,yum,yuzu,yve,yvette,za,zatarain,zatarains,zatarians,zeera,zero,zest,zesty,zinfandel,zinger,ziploc,ziti,zoom,zucchini,zwieback
137739,arriba baked winter squash mexican style,55.0,47892.0,2005-09-16,"[60-minutes-or-less, time-to-make, course, mai...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11.0,"[make a choice and proceed with recipe, depend...",autumn is my favorite time of year to cook! th...,"[winter squash, mexican seasoning, mixed spice...",7.0,51.5,0.0,13.0,0.0,2.0,0.0,4.0,make a choice and proceed with recipe\ndependi...,697.0,winter squash\nmexican seasoning\nmixed spice\...,60-minutes-or-less\ntime-to-make\ncourse\nmain...,winter squash mexican seasoning mixed spice ho...,winter squash mexican seasoning mixed spice ho...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
31490,a bit different breakfast pizza,30.0,26278.0,2002-06-17,"[30-minutes-or-less, time-to-make, course, mai...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9.0,"[preheat oven to 425 degrees f, press dough in...",this recipe calls for the crust to be prebaked...,"[prepared pizza crust, sausage patty, eggs, mi...",6.0,173.4,18.0,0.0,17.0,22.0,35.0,1.0,preheat oven to 425 degrees f\npress dough int...,386.0,prepared pizza crust\nsausage patty\neggs\nmil...,30-minutes-or-less\ntime-to-make\ncourse\nmain...,prepared pizza crust sausage patty eggs milk s...,prepared pizza crust sausage patty egg milk sa...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
112140,all in the kitchen chili,130.0,196586.0,2005-02-25,"[time-to-make, course, preparation, main-dish,...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6.0,"[brown ground beef in large pot, add chopped o...",this modified version of 'mom's' chili was a h...,"[ground beef, yellow onions, diced tomatoes, t...",13.0,269.8,22.0,32.0,48.0,39.0,27.0,5.0,brown ground beef in large pot\nadd chopped on...,293.0,ground beef\nyellow onions\ndiced tomatoes\nto...,time-to-make\ncourse\npreparation\nmain-dish\n...,ground beef yellow onions diced tomatoes tomat...,ground beef yellow onion dice tomato tomato pa...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.274827,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
59389,alouette potatoes,45.0,68585.0,2003-04-14,"[60-minutes-or-less, time-to-make, course, mai...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11.0,[place potatoes in a large pot of lightly salt...,"this is a super easy, great tasting, make ahea...","[spreadable cheese with garlic and herbs, new ...",11.0,368.1,17.0,10.0,2.0,14.0,8.0,20.0,place potatoes in a large pot of lightly salte...,543.0,spreadable cheese with garlic and herbs\nnew p...,60-minutes-or-less\n

In [13]:
my_data.iloc[231635]

name                         cookies by design   cookies on a stick
id                                                           298512
minutes                                                          29
contributor_id                                               506822
submitted                                       2008-04-15 00:00:00
tags              ['30-minutes-or-less', 'time-to-make', 'course...
nutrition                 [188.0, 11.0, 57.0, 11.0, 7.0, 21.0, 9.0]
n_steps                                                           9
steps             ['place melted butter in a large mixing bowl a...
description       i've heard of the 'cookies by design' company,...
ingredients       ['butter', 'eagle brand condensed milk', 'ligh...
n_ingredients                                                    10
Name: 231635, dtype: object